# Convert each value to array(window): this value and some next/previous
# if not enough next/prev -> ignore element
# returns smaller array -> expand arr before using this function

In [1]:
import numpy as np

def roll_arr_fwd_back1(arr, window_size, step=1, mode='back'):
    data = []
    for rowid in range(arr.shape[0]): # for in range(9): 0 1 ... 8
        if mode=='back':
            if rowid >= window_size-1: # if i=1, h_s=3 - not enough history; i=2 or 3 -> good
                good_rowids = [el for el in range(rowid-window_size+1, rowid+1, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
                data.append(arr[good_rowids])
        elif mode=='fwd':   
            if rowid <= arr.shape[0]-window_size: # if 9 rows: i=7, t_s=2 - not enough history; i=2 or 3 -> good
                good_rowids = [el for el in range(rowid, rowid+window_size, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
                data.append(arr[good_rowids]) #train_df.values[[0,1,2,3]] # get first 4 rows
    return np.array(data)

def roll_arr_same_size(arr, back_window, fwd_window, step=1, mode='back'):
    data = []
    #for rowid in range(arr.shape[0]): # for in range(9): 0 1 ... 8
    for i in range(back_window-1,arr.shape[0]-fwd_window+1): # for in range(9): 0 1 ... 8
        if mode=='back':
            good_rowids = [el for el in range(i-back_window+1, i+1, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
            data.append(arr[good_rowids])
        elif mode=='fwd':   
            good_rowids = [el for el in range(i, i+fwd_window, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
            data.append(arr[good_rowids]) #train_df.values[[0,1,2,3]] # get first 4 rows
    return np.array(data)

def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []; labels = []
    start_index = start_index + history_size
    if end_index is None: end_index = len(dataset) - target_size
    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])
        if single_step: labels.append(target[i+target_size])
        else: labels.append(target[i:i+target_size])
    return np.array(data), np.array(labels)


step = 1
past_history = 3
future_target = 2
x_train = np.array([
    [ 0,0.5],
    [-1,1.5],
    [-2,2.5],
    [-3,3.5],
    [-4,4.5],
])
y_train = np.array([
    [0],
    [1],
    [2],
    [3],
    [4],
])
dataset = np.concatenate((y_train, x_train), axis=1)


# tensorflow example

In [2]:
# X
multivariate_data(x_train, y_train, 0, x_train.shape[0], past_history, future_target, 1)[0]


array([[[ 0. ,  0.5],
        [-1. ,  1.5],
        [-2. ,  2.5]],

       [[-1. ,  1.5],
        [-2. ,  2.5],
        [-3. ,  3.5]]])

In [3]:
# Y
multivariate_data(x_train, y_train, 0, x_train.shape[0], past_history, future_target, 1)[1]


array([array([[3],
       [4]]), array([[4]])], dtype=object)

# Different result size

In [4]:
roll_arr_fwd_back1(arr=x_train, window_size=past_history, step=step, mode='back')


array([[[ 0. ,  0.5],
        [-1. ,  1.5],
        [-2. ,  2.5]],

       [[-1. ,  1.5],
        [-2. ,  2.5],
        [-3. ,  3.5]],

       [[-2. ,  2.5],
        [-3. ,  3.5],
        [-4. ,  4.5]]])

In [5]:
roll_arr_fwd_back1(arr=y_train, window_size=future_target, step=step, mode='fwd')


array([[[0],
        [1]],

       [[1],
        [2]],

       [[2],
        [3]],

       [[3],
        [4]]])

# same result len

In [6]:
roll_arr_same_size(arr=x_train, back_window=past_history, fwd_window=future_target, step=step, mode='back')


array([[[ 0. ,  0.5],
        [-1. ,  1.5],
        [-2. ,  2.5]],

       [[-1. ,  1.5],
        [-2. ,  2.5],
        [-3. ,  3.5]]])

In [7]:
roll_arr_same_size(arr=y_train, back_window=past_history, fwd_window=future_target, step=step, mode='fwd')


array([[[2],
        [3]],

       [[3],
        [4]]])